<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/code/content_analysis/bert/crime/bert/crime_annotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

In [ ]:
import os
import pandas as pd
import csv
import sys
import numpy as np


# parameters
clsfr_path = "drive/MyDrive/Bild/crime_clsfr_BERT_torch"
input_path = "https://www.dropbox.com/s/ousyjlju18fopny/_migration_articles_BERT.csv?dl=1"
output_path = "drive/MyDrive/Bild/crime_annotations.csv"
batch_size = 16

# load classifier and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
model = AutoModelForSequenceClassification.from_pretrained(clsfr_path)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-german-cased', model_max_length = 512)
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device = 0, padding='max_length', truncation=True)

Sanity Check:

In [4]:
# Test model
text = "Die Polizei hat gestern zwei Flüchtige nach einem Bankraub festgenommen. Nach einem kurzen Schusswechsel konnten die mutmaßlichen Täter überwältigt und festegnommen werden. Die Staatsanwaltschaft ermittelt."
classifier(text)

[{'label': 'LABEL_1', 'score': 0.9996830224990845}]

In [5]:
text = "Schleswig-Holstein hat einen neuen Landtag gewählt. Mit dem vorläufigen Endergebnis kommt die CDU als stärkste Kraft auf 43,4 Prozent der Stimmen, die SPD erreicht 16 Prozent, die Grünen sind zweitstärkste Kraft mit 18,3 Prozent."
classifier(text)

[{'label': 'LABEL_0', 'score': 0.9996991157531738}]

Looking good.

In [6]:
mig_articles = pd.read_csv(input_path)

In [7]:
mig_articles.head()

,date,paper,title,url,text
0,15707,taz,Ländle ärgert sich über Lästerei,https://taz.de/Archiv-Suche/!527952&s=&SuchRah...,BERLIN | Nach den Lästereien von Bundestagsviz...
1,15707,taz,Die Kinder bleiben unter sich,https://taz.de/Archiv-Suche/!5076321&s=/,QABB ELIAS taz | Der elfjährige Firas wird vie...
2,15707,taz,Auf der Suche nach Vorbildern,https://taz.de/Archiv-Suche/!528028&s=&SuchRah...,Als „sehr positiv“ bewertet der Senat die Erfa...
3,15707,taz,Wenig arbeitslose Jugendliche,https://taz.de/Archiv-Suche/!528056&s=&SuchRah...,Die Jugendarbeitslosigkeit scheint in Niedersa...
4,15706,taz,„Jesus setzte sich für Fremde ein“,https://taz.de/Archiv-Suche/!5076305&s=/,Mehrere Dutzend Personen eines Kollektivs zur ...


In [8]:
mig_articles["crime"] = classifier([article for article in mig_articles.text])

In [9]:
import re
int(re.match('LABEL_(\d)', 'LABEL_0').group(1))


0

In [10]:
# clean
labels = []
prob = []
for dt in mig_articles.crime:
  labels.append(int(re.match('LABEL_(\d)', dt['label']).group(1)))
  prob.append(float(dt['score']))

mig_articles['crime_label'] = labels
mig_articles['crime_prob'] = prob

In [11]:
mig_articles.to_csv(output_path)

In [12]:
mig_articles.head()

,date,paper,title,url,text,crime,crime_label,crime_prob
0,15707,taz,Ländle ärgert sich über Lästerei,https://taz.de/Archiv-Suche/!527952&s=&SuchRah...,BERLIN | Nach den Lästereien von Bundestagsviz...,"{'label': 'LABEL_0', 'score': 0.9999176263809204}",0,0.999918
1,15707,taz,Die Kinder bleiben unter sich,https://taz.de/Archiv-Suche/!5076321&s=/,QABB ELIAS taz | Der elfjährige Firas wird vie...,"{'label': 'LABEL_0', 'score': 0.9999104738235474}",0,0.999910
2,15707,taz,Auf der Suche nach Vorbildern,https://taz.de/Archiv-Suche/!528028&s=&SuchRah...,Als „sehr positiv“ bewertet der Senat die Erfa...,"{'label': 'LABEL_0', 'score': 0.9998176693916321}",0,0.999818
3,15707,taz,Wenig arbeitslose Jugendliche,https://taz.de/Archiv-Suche/!528056&s=&SuchRah...,Die Jugendarbeitslosigkeit scheint in Niedersa...,"{'label': 'LABEL_0', 'score': 0.9997981190681458}",0,0.999798
4,15706,taz,„Jesus setzte sich für Fremde ein“,https://taz.de/Archiv-Suche/!5076305&s=/,Mehrere Dutzend Personen eines Kollektivs zur ...,"{'label': 'LABEL_0', 'score': 0.993591845035553}",0,0.993592
